In [ ]:
#| default_exp test_utils

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
import numpy as np, re, inspect
from xcai.data import *
from xcai.transform import *
from fastcore.meta import *
from typing import Optional, Dict
from transformers import AutoTokenizer, BatchEncoding

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

## Config

In [ ]:
#| export
PARAM = {
    'cols': ['identifier', 'input_text'],
    'use_tokz': True,
    'tokz': 'bert-base-cased',
    'fld': 'input_text',
    'max_len': 32,
    'pad_side': 'right',
    'inp': 'data',
    'targ': 'lbl2data',
    'ptr': 'lbl2data_data2ptr',
    'drop': True,
    'ret_t': True,
    'in_place': True,
    'collapse': True,
    'device': 'cpu',
}

In [ ]:
#| export
TRAIN_CFG = {
    'path': {
        'train': {
            'data_lbl': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/trn_X_Y.txt',
            'data_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/train.raw.txt',
            'lbl_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/label.raw.txt',
            'data_lbl_filterer': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/filter_labels_train.txt',
        },
    },
    'parameters': PARAM,
}

In [ ]:
#| export
DATA_CFG = {
    'path': {
        'train': {
            'data_lbl': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/trn_X_Y.txt',
            'data_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/train.raw.txt',
            'lbl_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/label.raw.txt',
            'data_lbl_filterer': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/filter_labels_train.txt',
        },
        'test': {
            'data_lbl': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/tst_X_Y.txt',
            'data_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/test.raw.txt',
            'lbl_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/label.raw.txt',
            'data_lbl_filterer': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/filter_labels_test.txt',
        },
    },
    'parameters': PARAM,
}

In [ ]:
#| export
TRAIN_META_CFG = {
    'path': {
        'train': {
            'data_lbl': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/trn_X_Y.txt',
            'data_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/train.raw.txt',
            'lbl_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/label.raw.txt',
            'data_lbl_filterer': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/filter_labels_train.txt',
            'hlk_meta': {
                'prefix': 'hlk',
                'data_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_trn_X_Y.txt',
                'lbl_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_lbl_X_Y.txt',
                'meta_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/hyper_link.raw.txt'
            },
        },
    },
    'parameters': PARAM,
}

In [ ]:
#| export
DATA_META_CFG = {
    'path': {
        'train': {
            'data_lbl': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/trn_X_Y.txt',
            'data_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/train.raw.txt',
            'lbl_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/label.raw.txt',
            'data_lbl_filterer': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/filter_labels_train.txt',
            'hlk_meta': {
                'prefix': 'hlk',
                'data_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_trn_X_Y.txt',
                'lbl_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_lbl_X_Y.txt',
                'meta_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/hyper_link.raw.txt'
            },
        },
        'test': {
            'data_lbl': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/tst_X_Y.txt',
            'data_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/test.raw.txt',
            'lbl_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/label.raw.txt',
            'data_lbl_filterer': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/filter_labels_test.txt',
            'hlk_meta': {
                'prefix': 'hlk',
                'data_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_tst_X_Y.txt',
                'lbl_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_lbl_X_Y.txt',
                'meta_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/hyper_link.raw.txt',
            },
        },
    },
    'parameters': PARAM,
}

## Tester

In [ ]:
#| export
CFGS = {'train':TRAIN_CFG, 'data':DATA_CFG, 'data_meta':DATA_META_CFG, 'train_meta':TRAIN_META_CFG}

In [ ]:
#| export
class XCBlock:

    @delegates(XCDataBlock.from_cfg)
    @classmethod
    def from_cfg(cls, cfg:str, bsz:Optional[int]=10, **kwargs):
        if cfg in CFGS: cfg = CFGS[cfg]
        else: raise ValueError(f"Invalid `cfg` value {cfg}, it should be one of ['train', 'data', 'train_meta', 'data_meta']")
        for k in cfg['parameters']: 
            if k in kwargs and kwargs[k] is not None: cfg['parameters'][k]=kwargs.pop(k)
        tokz = AutoTokenizer.from_pretrained(cfg['parameters']['tokz'])
        cfg['parameters']['sep_tok'], cfg['parameters']['pad_tok'], cfg['parameters']['batch_size'] = tokz.sep_token_id, tokz.pad_token_id, bsz
        collator = XCCollator(TfmPipeline([XCPadFeatTfm(**cfg['parameters']), AlignInputIdsTfm(**cfg['parameters'])]))
        return XCDataBlock.from_cfg(cfg, collate_fn=collator, **kwargs)


### Example

In [ ]:
block = XCBlock.from_cfg('train', tokz='bert-base-uncased')

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
b = block.train.one_batch()

In [ ]:
b

{'lbl2data_idx': tensor([300904, 177733,  19097, 133569, 133575, 105663, 105665, 124710, 125937,
        166914, 166916, 193822, 216272,  16898,  56577,  17888,  57311, 122156,
        128004, 171682,  98534]), 'lbl2data_identifier': ['List_of_fossiliferous_stratigraphic_units_in_Illinois', 'Culture_of_Papua_New_Guinea', 'History_of_Mexico', 'La_Gran_Chichimeca', 'Nueva_Galicia', 'APMonitor', 'GEKKO', 'MATLAB', 'Modelica', 'ASCEND', 'EMSO', 'Serbian_SuperLiga', 'List_of_football_stadiums_in_Serbia', 'List_of_islands_of_Norway', 'Communes_of_the_Savoie_department', 'Highland_dress', 'Force-fire', 'St_Kilda', 'Rockall', 'List_of_Western_Australia_first-class_cricketers', 'List_of_municipalities_and_towns_in_Slovakia'], 'lbl2data_input_text': ['List of fossiliferous stratigraphic units in Illinois', 'Culture of Papua New Guinea', 'History of Mexico', 'La Gran Chichimeca', 'Nueva Galicia', 'APMonitor', 'GEKKO', 'MATLAB', 'Modelica', 'ASCEND', 'EMSO', 'Serbian SuperLiga', 'List of football 

In [ ]:
b = block.train.dset.one_batch()

In [ ]:
b

[{'data_identifier': 'Ferron_Formation',
  'data_input_text': 'Ferron Formation',
  'data_input_ids': [101, 10768, 18933, 2078, 4195, 102],
  'data_token_type_ids': [0, 0, 0, 0, 0, 0],
  'data_attention_mask': [1, 1, 1, 1, 1, 1],
  'lbl2data_idx': [246699, 246701],
  'lbl2data_identifier': ['List_of_fossiliferous_stratigraphic_units_in_Utah',
   'Paleontology_in_Utah'],
  'lbl2data_input_text': ['List of fossiliferous stratigraphic units in Utah',
   'Paleontology in Utah'],
  'lbl2data_input_ids': [[101,
    2862,
    1997,
    10725,
    23930,
    2358,
    8609,
    8004,
    20721,
    3197,
    1999,
    6646,
    102],
   [101, 5122, 12162, 6779, 1999, 6646, 102]],
  'lbl2data_token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0]],
  'lbl2data_attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1, 1, 1]]},
 {'data_identifier': 'Jet_(lignite)',
  'data_input_text': 'Jet (lignite)',
  'data_input_ids': [101, 6892, 1006, 5622, 29

## Batch

In [ ]:
#| export
def prepare_batch(m, b, m_args=None):
    m_kwargs = inspect.signature(m.forward).parameters
    return BatchEncoding({k:v for k,v in b.items() if k in m_kwargs or (m_args is not None and k in m_args)})